In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS #Lista predeterminada de palabras vacías en inglés

In [2]:
data=pd.read_csv("./deteccion_spam/datos/enronSpamSubset.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  10000 non-null  int64 
 1   Unnamed: 0    10000 non-null  int64 
 2   Body          10000 non-null  object
 3   Label         10000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 312.6+ KB


In [4]:
def preprocesamiento(data):
    data.drop_duplicates() #Eliminamos los correos duplicados (parece que no había)
    data.dropna() #Eliminamos los correos con valores nulos (parece que no hay)
    
    #División del conjunto de datos en entrenamiento y prueba
    X= data["Body"] #Atributos (sólo hay uno)
    y= data["Label"] #Etiquetas
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42) #Dividimos en conjunto de entrenamiento y de prueba (20% prueba)
    
    #Creamos una lista personalizada de palabras vacías
    stop_words_list = list(ENGLISH_STOP_WORDS)
    stop_words_list += ["subject"]

    #TOKENIZAMOS (lo hacemos fuera pq luego voy a utilizar el vectorizer)
    vectorizer=CountVectorizer(min_df=2, stop_words=stop_words_list)
    X_train=vectorizer.fit_transform(X_train)
    X_test=vectorizer.transform(X_test)
    print(vectorizer.vocabulary_)
    
    return X_train, X_test, y_train, y_test, vectorizer

In [5]:
X_train, X_test, y_train, y_test, vectorizer= preprocesamiento(data)

{'intrastate': 20021, '311': 912, 'contracts': 10125, 'guadalupe': 17531, 'pipeline': 26856, 'daren': 11236, 'need': 24463, 'contact': 10043, 'credit': 10700, 'department': 11858, 'denver': 11848, 'andy': 3924, 'walls': 36611, '720': 1860, '944': 2300, '9541': 2321, 'manager': 22578, 'chris': 8610, 'farmer': 15305, 'enron': 14253, 'com': 9333, 'duke': 13265, 'energy': 14164, 'cc': 8042, '01': 33, '07': 128, '02': 50, 'pm': 27057, 'requirements': 29516, 'new': 24595, 'entity': 14307, 'rating': 28687, 'original': 25564, 'message': 23325, 'christopher': 8627, 'sent': 31181, 'friday': 16403, 'january': 20396, '04': 81, '2002': 612, '52': 1419, 'brian': 6897, 'request': 29506, 'marketer': 22733, 'attached': 4868, 'contract': 10119, 'forms': 16173, 'file': 15618, 'form': 16151, 'transportation': 34953, 'doc': 12804, 'exhibit': 14927, 'intra': 20015, '31': 906, 'agreement': 3246, 'questions': 28463, 'senior': 31163, 'analyst': 3868, '713': 1845, '627': 1657, 'mail': 22479, 'property': 27989, 

In [6]:
X_train

<8000x37801 sparse matrix of type '<class 'numpy.int64'>'
	with 684097 stored elements in Compressed Sparse Row format>

In [ ]:
import operator

# Obtener el vocabulario y sus recuentos de palabras
vocab = vectorizer.vocabulary_
word_counts = {word: X_train.getcol(idx).sum() for word, idx in vocab.items()}

# Ordenar el vocabulario por frecuencia de palabra (de mayor a menor)
sorted_vocab = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

# Imprimir las palabras ordenadas por frecuencia
print("Palabras ordenadas por frecuencia:")
for word, count in sorted_vocab:
    print(word, count)

In [7]:
#Función donde entrenamos el clasificador Naive Bayes Multinomial
def clasificador_NBMultinomial(X_train, y_train):
    classifier=MultinomialNB()
    classifier.fit(X_train,y_train)
    
    return classifier

In [8]:
#Función que devuelve las medidas del clasificador
def medidas_NBMultinomial(X_test,y_test):
    accuracy=classifier.score(X_test,y_test)
    medidas=accuracy #añadir más
    return medidas

In [9]:
#Función para hacer nuevas predicciones con el modelo entrenado
def prediccion(new_emails, classifier, vectorizer):
    new_emails_transformed=vectorizer.transform(new_emails)
    predictions=classifier.predict(new_emails_transformed)
    return predictions

In [10]:
new_emails=[
    "Give me 5000$ or I will publish your nudes",
    "Pass by my office so we can discuss about your new assignment",
    "hello",
    "enron",
    "refrigera",
    "londrina"
    ]

In [11]:
classifier=clasificador_NBMultinomial(X_train, y_train)
medidas=medidas_NBMultinomial(X_test,y_test)
print(medidas)
nueva_prediccion=prediccion(new_emails, classifier, vectorizer)
print(nueva_prediccion)

0.984
[1 0 1 0 0 0]
